<a href="https://colab.research.google.com/github/ewattudo/gis/blob/main/PS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-Up & Import Files
Run this for data and setup code.



## Set Up

In [3]:
%%capture
!pip install geopandas==1.0.1
!pip install mapclassify

In [4]:
import os, zipfile #basics
import pandas as pd #data management
import matplotlib.pyplot as plt #vis

import geopandas as gpd #gis/maps

#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from google.colab import files #to download from colab onto hd

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

In [5]:
#!python --version
gpd.__version__

'1.0.1'

In [6]:
!pip install mapclassify
import mapclassify #need for thematic map classification